In [ ]:
import pandas as pd
import requests, time, os, copy, re, random, uuid, subprocess
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from time import strptime
from datetime import datetime
import urllib.request
from newspaper import Config, Article, Source
from joblib import Parallel, delayed
from tqdm import tqdm, trange
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
def wait_for_internet_connection():
    while True:
        try:
            response = urllib.request.urlopen('https://facebook.com',timeout=1)
            return
        except urllib.request.URLError:
            pass
        
def pause():
    programPause = input("Press the <ENTER> key to continue...")

# Forbes

### a. Scrape URL's from Google Search

In [ ]:
chromedriver = '/Users/elipickh/Downloads/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

#chrome_options = Options()
#adblock:
#chrome_options.add_extension('/Users/elipickh/Downloads/extension_1_13_3.crx')
#chrome_options.add_argument('headless')
#driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

driver = webdriver.Chrome(chromedriver)                                    

appended_data = []
appended_data_0 = []
appended_data_1 = []

first = True
bot_warn = 'Our systems have detected unusual traffic from your computer network'

for x in range(0, 1):
#for x in range(0, 2):    
    for y in range(4, 8):
        for m in range (1, 13):
            for d in range(1, 32):
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=' + str(uuid.uuid4()))
                #time.sleep(random.uniform(1, 1.5))
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=site:www.forbes.com/sites/*/201' + str(y) +'/' + str('{0:02}'.format(m)) + '/' + str('{0:02}'.format(d)) + '/' +'&filter=' + str(x))
                src = driver.page_source

                if first:
                    pause()
                    first = False
                elif bool(re.search(bot_warn, src)):
                    os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'")
                    pause()
                #else:
                    #time.sleep(random.uniform(61, 63))
                    #time.sleep(random.uniform(1, 1.5))

                while True:
                    for i in range(1, 150):
                        try:
                            
                            url = driver.find_element_by_css_selector('.g:nth-child(' + str(i) +") a").get_attribute('href')
                            appended_data.append(url)
                            
                            if x == 0:
                                appended_data_0.append(url)
                            if x == 1:
                                appended_data_1.append(url)

                        except:
                            #print("no more links")
                            break
                    #time.sleep(random.uniform(0.5, 1))

                    try: 
                        # this is navigate to next page 
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(random.uniform(0.5, 1))
                        button = driver.find_element_by_css_selector('#pnnext .ch+ span')
                        wait_for_internet_connection()
                        button.click() 
                        #time.sleep(random.uniform(2, 3))
                    except: 
                        #print("no more pages")
                        break
                if x == 0:
                    print(str(len(set(appended_data_0))) + '  ' + str(m) + '/' + str(d) + '--' + str(y))
                if x == 1:
                    print(str(len(set(appended_data_1))) + '  ' + str(m) + '/' + str(d) + '--' + str(y))


# only in filter=0
print(len(list(set(appended_data_0).difference(appended_data_1))))
# only in filter=1
print(len(list(set(appended_data_1).difference(appended_data_0))))
# in both
print(len(list(set(appended_data_0).intersection(appended_data_1))))

appended_data2 = set(appended_data)

In [ ]:
df = pd.DataFrame(appended_data_0, columns=["colummn"])
df.to_csv('forbes_list.csv', index=False)

In [ ]:
forbes_data = pd.read_csv("forbes_list.csv", header=0)
forbes_urls = list(forbes_data.colummn)
len(set(forbes_urls))

In [ ]:
forbes_urls = ["/".join(x.split("/")[:9]) for x in forbes_urls]
forbes_urls = [x+'/' for x in forbes_urls]
forbes_urls = [x.replace('http:/','https:/') for x in forbes_urls]
forbes_urls = list(set(forbes_urls))
len(forbes_urls)

### b. Fetch article content from URL's

In [ ]:
config = Config()
config.MAX_TEXT = 1000000

def chunkify(lst,n):
    return [ lst[i::n] for i in range(n) ]

def process_article(url):
    for x in range(0, 5):
        try:
            article = Article(url, config)
            article.download()
            article.parse()

            headline = article.title
            publish_date = article.publish_date
            authors = article.authors
            fulltext = article.text
            article.nlp()
            keywords = article.keywords
            summary = article.summary
            forbes_all = {'url':url, 
                          'date':publish_date, 
                          'authors':authors, 
                          'headline':headline, 
                          'fulltext':fulltext, 
                          'summary':summary, 
                          'keywords':keywords}
            return forbes_all
        except:
            wait_for_internet_connection()
            time.sleep(0.1)
            continue
        break

In [ ]:
results = Parallel(n_jobs=-1)(delayed(process_article)(url) for url in tqdm(forbes_urls))

results = [x for x in results if x is not None]
results_df = pd.DataFrame(results)
results_df.to_csv('forbes_output.csv', index=False)

In [ ]:
forbes_output = pd.read_csv("forbes_output.csv", header=0)
forbes_output.drop('summary', axis=1, inplace=True)
forbes_output_list = forbes_output.to_dict('records')

In [ ]:
def clean_text(s):
    return str(s.encode('ascii','ignore'), 'utf-8').replace('\n', ' ')

for d in forbes_output_list:
    for k, v in d.items():
        if k=='fulltext' or k=='headline':
            d[k]=clean_text(v)

In [ ]:
for d in forbes_output_list:
    for k, v in list(d.items()):
        if k=='fulltext':
            d['sentences']=tokenize.sent_tokenize(v)

In [ ]:
sid = SentimentIntensityAnalyzer()

for d in forbes_output_list:
    for k, v in list(d.items()):
        if k=='headline':
            #for v in v:
            d['headline_score']=sid.polarity_scores(v)
        if k=='sentences':
            a=[]
            for v in v:
                a.append(sid.polarity_scores(v))  
                d['fulltext_scores'] = a

In [ ]:
import numpy as np

for maindict in forbes_output_list:
    compound_list = []
    pos_list =[]
    neg_list =[]
    neu_list =[]
    compound_list_nonzero = []
    pos_list_nonzero =[]
    neg_list_nonzero =[]
    neu_list_nonzero =[]    
    for subdict in maindict['fulltext_scores']:
        compound_list.append(subdict['compound'])
        pos_list.append(subdict['pos'])
        neg_list.append(subdict['neg'])
        neu_list.append(subdict['neu'])
        if subdict['compound']!=0:
            compound_list_nonzero.append(subdict['compound'])
        if subdict['pos']!=0:
            pos_list_nonzero.append(subdict['pos'])
        if subdict['neg']!=0:
            neg_list_nonzero.append(subdict['neg'])
        if subdict['neu']!=0:
            neu_list_nonzero.append(subdict['neu'])  
    
    maindict['mean_compound'] = np.mean(compound_list)
    maindict['median_compound'] = np.median(compound_list)
    maindict['mean_pos'] = np.mean(pos_list)
    maindict['median_pos'] = np.median(pos_list)
    maindict['mean_neg'] = np.mean(neg_list)
    maindict['median_neg'] = np.median(neg_list)
    maindict['mean_neu'] = np.mean(neu_list)
    maindict['median_neu'] = np.median(neu_list)
    
    maindict['mean_compound_nonzero'] = np.mean(compound_list_nonzero)
    maindict['median_compound_nonzero'] = np.median(compound_list_nonzero)
    maindict['mean_pos_nonzero'] = np.mean(pos_list_nonzero)
    maindict['median_pos_nonzero'] = np.median(pos_list_nonzero)
    maindict['mean_neg_nonzero'] = np.mean(neg_list_nonzero)
    maindict['median_neg_nonzero'] = np.median(neg_list_nonzero)
    maindict['mean_neu_nonzero'] = np.mean(neu_list_nonzero)
    maindict['median_neu_nonzero'] = np.median(neu_list_nonzero)

In [ ]:
df2 = pd.DataFrame(forbes_output_list)
df2.to_csv('forbes_output_list_1.csv', index=False)

### c. Get page views and categories for each article

In [ ]:
# Doing below in Ubuntu EC2
# Block the following domains from, by entering: sudo nano /etc/hosts
# and then pasting these lines in the bottom: (don't include facebook, as it's used to test for internet)
127.0.0.1 bit.ly
127.0.0.1 truste.com
127.0.0.1 trefis.com
127.0.0.1 twitter.com
127.0.0.1 google.com
127.0.0.1 google-analytics.com
127.0.0.1 googletagmanager.com
127.0.0.1 doubleclick.net
127.0.0.1 youtube.com
127.0.0.1 googlesyndication.com
127.0.0.1 brightcove.com
127.0.0.1 media.net
127.0.0.1 www.bit.ly
127.0.0.1 www.truste.com
127.0.0.1 www.trefis.com
127.0.0.1 www.twitter.com
127.0.0.1 www.google.com
127.0.0.1 www.google-analytics.com
127.0.0.1 www.googletagmanager.com
127.0.0.1 www.doubleclick.net
127.0.0.1 www.youtube.com
127.0.0.1 www.googlesyndication.com
127.0.0.1 www.brightcove.com
127.0.0.1 www.media.net
#Make sure it's saved, then restart Ubuntu
# if need to close many open Chrome windwos, run: killall chrome 

In [ ]:
chrome_options = Options()

prefs = {'profile.managed_default_content_settings.images': 2}
chrome_options.add_experimental_option('prefs', prefs)

# uBlock Origin
chrome_options.add_extension('/Users/elipickh/Downloads/extension_1_13_2.crx')
# Forbes Splash Screen Bypass
chrome_options.add_extension('/Users/elipickh/Downloads/extension_0_1.crx')
chrome_options.add_argument('--dns-prefetch-disable')
#chrome_options.add_argument('headless')
chrome_options.add_argument('window-position=0,0')
# Make sure this restricted size below doesn't intefere/change the way items load!
chrome_options.add_argument('window-size=600,800')

chromedriver = '/Users/elipickh/Downloads/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

#driver.implicitly_wait(0.5)

def driver_article(url):
    for x in range(0, 2):
        try:
            driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)
            driver.get(url)
            views = driver.find_element_by_css_selector('.view-count').text
            main_cat = driver.find_element_by_css_selector('.crumb').text
            try:
                sub_cat = driver.find_element_by_css_selector('.hashtag').text
            except:
                sub_cat = ''
            forbes_output_selenium = {'url':url, 'views':views, 'main_cat':main_cat, 'sub_cat':sub_cat}
            driver.close()
            return forbes_output_selenium
        except:
            pass
        finally:
            try:
                driver.close()
            except:
                pass


forbes_urls.sort()
FNULL = open(os.devnull, 'w')
os.chdir('home/ubuntu/Desktop/output/')
for i in trange(0, 200):
    # n_jobs=36 is fastest, but might crash. 12 is safer
    results = Parallel(n_jobs=36)(delayed(driver_article)(url) for url in tqdm(chunkify(forbes_urls, 200)[i]))
    time.sleep(1)
    results = [x for x in results if x is not None]
    results_df = pd.DataFrame(results)
    time.sleep(1)
    results_df.to_csv('forbes_selenium_output' + str(i) + '.csv', index = False)
    a = subprocess.call('killall -9 chrome', stdout = FNULL, stedrr = subprocess.STDOUT, shell = True)
    time.sleep(5)

In [ ]:
articles_final = pd.read_csv('forbes_output_list_1.csv')
articles_final = articles_final.drop_duplicates()

In [ ]:
views_final = pd.read_csv("views_final.csv", thousands = ',')
views_final= views_final.drop_duplicates()

In [ ]:
forbes_combined_output = articles_final.merge(views_final, on='url', how = 'left')
forbes_combined_output = forbes_combined_output.drop_duplicates()

In [ ]:
print(len(forbes_combined_output))
print(len(articles_final))
print(len(views_final))

In [ ]:
forbes_combined_output.to_csv('forbes_combined_output.csv.gz', index = False, compression='gzip')

# CNN URL's and article content

In [ ]:
chrome_options = Options()
#chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
prefs = {'profile.managed_default_content_settings.images': 2}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--dns-prefetch-disable')
chromedriver = '/Users/elipickh/Downloads/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver) 

appended_data = []
appended_data_0 = []
appended_data_1 = []


first = True
bot_warn = 'Our systems have detected unusual traffic from your computer network'

#for x in range(0, 2):
for x in range(0, 1):    
    for y in range(4, 8):
        for m in range (1, 13):
            for d in range(1, 32):
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=' + str(uuid.uuid4()))
                #time.sleep(random.uniform(1, 1.5))
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=site:cnn.com/201' + str(y) +'/' + str('{0:02}'.format(m)) + '/' + str('{0:02}'.format(d)) + '/' +'&filter=' + str(x))
                src = driver.page_source

                if first:
                    pause()
                    first = False
                elif bool(re.search(bot_warn, src)):
                    os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'")
                    pause()
                #else:
                    #time.sleep(random.uniform(61, 63))
                    #time.sleep(random.uniform(1, 1.5))

                while True:
                    for i in range(1, 150):
                        try:
                            
                            url = driver.find_element_by_css_selector('.g:nth-child(' + str(i) +") a").get_attribute('href')
                            appended_data.append(url)

                        except:
                            #print("no more links")
                            break
                    #time.sleep(random.uniform(0.5, 1))

                    try: 
                        # this is navigate to next page 
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(random.uniform(0.5, 1))
                        button = driver.find_element_by_css_selector('#pnnext .ch+ span')
                        wait_for_internet_connection()
                        button.click() 
                        #time.sleep(random.uniform(2, 3))
                    except: 
                        #print("no more pages")
                        break
                print(str(len(set(appended_data))) + '  ' + str(m) + '/' + str(d) + '--' + str(y))



appended_data = set(appended_data)

In [ ]:
cnn_urls = [s for s in appended_data if "cnnespanol.cnn.com" not in s]
cnn_urls = [x.split('index.htm', 1)[0] for x in cnn_urls]
cnn_urls = [x.split('?', 1)[0] for x in cnn_urls]
cnn_urls = [x.split('comment-page-', 1)[0] for x in cnn_urls]
cnn_urls = [x.replace('https:/','http:/') for x in cnn_urls]
cnn_urls = [x.replace('www.','') for x in cnn_urls]
cnn_urls = list(set(cnn_urls))
print(len(cnn_urls))

In [ ]:
df = pd.DataFrame(cnn_urls, columns=["column"])
df.to_csv('cnn_list.csv', index=False)

In [ ]:
cnn_data = pd.read_csv("cnn_list.csv", header=0)
cnn_urls = list(cnn_data.column)
cnn_urls = [x.split('%', 1)[0] for x in cnn_urls]
len(set(cnn_urls))

In [ ]:
config = Config()
config.MAX_TEXT = 1000000

def process_article(url):
    for x in range(0, 5):
        try:
            article = Article(url, config)
            article.download()
            article.parse()

            headline = article.title
            publish_date = article.publish_date
            authors = article.authors
            fulltext = article.text
            article.nlp()
            keywords = article.keywords
            #summary = article.summary
            cnn_all = {'url':url, 
                          'date':publish_date, 
                          'authors':authors, 
                          'headline':headline, 
                          'fulltext':fulltext, 
                          #'summary':summary, 
                          'keywords':keywords}
            return cnn_all
        except:
            wait_for_internet_connection()
            time.sleep(0.1)
            continue
        break



results = Parallel(n_jobs=-1)(delayed(process_article)(url) for url in tqdm(cnn_urls))

results = [x for x in results if x is not None]
results_df = pd.DataFrame(results)
results_df.to_csv('cnn_output.csv', index=False)

In [ ]:
cnn_output = pd.read_csv("cnn_output.csv", header=0)
cnn_output_list = cnn_output.to_dict('records')

In [ ]:
def clean_text(s):
    return str(str(s).encode('ascii','ignore'), 'utf-8').replace('\n', ' ')

for d in cnn_output_list:
    for k, v in d.items():
        if k=='fulltext' or k=='headline':
            d[k]=clean_text(v)

for d in cnn_output_list:
    for k, v in list(d.items()):
        if k=='fulltext':
            d['sentences']=tokenize.sent_tokenize(v)

for d in cnn_output_list:
    for k, v in list(d.items()):
        if k=='headline':
            d['headline_score']=sid.polarity_scores(v)
        if k=='sentences':
            a=[]
            for v in v:
                a.append(sid.polarity_scores(v))  
                d['fulltext_scores'] = a

In [ ]:
for maindict in cnn_output_list:
    compound_list = []
    pos_list =[]
    neg_list =[]
    neu_list =[]
    compound_list_nonzero = []
    pos_list_nonzero =[]
    neg_list_nonzero =[]
    neu_list_nonzero =[]    
    for subdict in maindict['fulltext_scores']:
        compound_list.append(subdict['compound'])
        pos_list.append(subdict['pos'])
        neg_list.append(subdict['neg'])
        neu_list.append(subdict['neu'])
        if subdict['compound']!=0:
            compound_list_nonzero.append(subdict['compound'])
        if subdict['pos']!=0:
            pos_list_nonzero.append(subdict['pos'])
        if subdict['neg']!=0:
            neg_list_nonzero.append(subdict['neg'])
        if subdict['neu']!=0:
            neu_list_nonzero.append(subdict['neu'])  
    
    maindict['mean_compound'] = np.mean(compound_list)
    maindict['median_compound'] = np.median(compound_list)
    maindict['mean_pos'] = np.mean(pos_list)
    maindict['median_pos'] = np.median(pos_list)
    maindict['mean_neg'] = np.mean(neg_list)
    maindict['median_neg'] = np.median(neg_list)
    maindict['mean_neu'] = np.mean(neu_list)
    maindict['median_neu'] = np.median(neu_list)
    
    maindict['mean_compound_nonzero'] = np.mean(compound_list_nonzero)
    maindict['median_compound_nonzero'] = np.median(compound_list_nonzero)
    maindict['mean_pos_nonzero'] = np.mean(pos_list_nonzero)
    maindict['median_pos_nonzero'] = np.median(pos_list_nonzero)
    maindict['mean_neg_nonzero'] = np.mean(neg_list_nonzero)
    maindict['median_neg_nonzero'] = np.median(neg_list_nonzero)
    maindict['mean_neu_nonzero'] = np.mean(neu_list_nonzero)
    maindict['median_neu_nonzero'] = np.median(neu_list_nonzero)

In [ ]:
df2 = pd.DataFrame(cnn_output_list)
df2.to_csv('cnn_output_list_1.csv', index=False)

# NYTimes URL's and article content

In [ ]:
chrome_options = Options()
#chrome_options.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
prefs = {'profile.managed_default_content_settings.images': 2}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--dns-prefetch-disable')
chromedriver = '/Users/elipickh/Downloads/chromedriver'
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver) 

appended_data = []
appended_data_0 = []
appended_data_1 = []

first = True
bot_warn = 'Our systems have detected unusual traffic from your computer network'

#for x in range(0, 2):
for x in range(0, 1):    
    for y in range(4, 8):
        for m in range (1, 13):
            for d in range(1, 32):
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=' + str(uuid.uuid4()))
                #time.sleep(random.uniform(1, 1.5))
                wait_for_internet_connection()
                driver.get('https://www.google.com/search?q=site:nytimes.com/201' + str(y) +'/' + str('{0:02}'.format(m)) + '/' + str('{0:02}'.format(d)) + '/' +'&filter=' + str(x))
                src = driver.page_source

                if first:
                    pause()
                    first = False
                elif bool(re.search(bot_warn, src)):
                    os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'"); time.sleep(1); os.system("printf '\a'")
                    pause()
                #else:
                    #time.sleep(random.uniform(61, 63))
                    #time.sleep(random.uniform(1, 1.5))

                while True:
                    for i in range(1, 150):
                        try:
                            
                            url = driver.find_element_by_css_selector('.g:nth-child(' + str(i) +") a").get_attribute('href')
                            appended_data.append(url)

                        except:
                            #print("no more links")
                            break
                    #time.sleep(random.uniform(0.5, 1))

                    try: 
                        # this is navigate to next page 
                        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(random.uniform(0.5, 1))
                        button = driver.find_element_by_css_selector('#pnnext .ch+ span')
                        wait_for_internet_connection()
                        button.click() 
                        #time.sleep(random.uniform(2, 3))
                    except: 
                        #print("no more pages")
                        break
                print(str(len(set(appended_data))) + '  ' + str(m) + '/' + str(d) + '--' + str(y))



appended_data = set(appended_data)

In [ ]:
nyt_urls = [x.replace('http:/','https:/') for x in appended_data]
nyt_urls = [x.replace('mobile.nytimes.com','nytimes.com') for x in nyt_urls]
nyt_urls = [x.replace('mwr.gtm.nytimes.com','nytimes.com') for x in nyt_urls]
nyt_urls = [x.replace('mwr.nytimes.com','nytimes.com') for x in nyt_urls]
nyt_urls = [x.replace('www.','') for x in nyt_urls]
nyt_urls = list(set(nyt_urls))
print(len(nyt_urls))

In [ ]:
df = pd.DataFrame(nyt_urls, columns=["column"])
df.to_csv('nyt_list.csv', index=False)

In [ ]:
#(fetched articles in EC2... which gave nyt_output.csv)

In [ ]:
nyt_output = pd.read_csv("nyt_output.csv", header=0)
nyt_output_list = nyt_output.to_dict('records')

In [ ]:
sid = SentimentIntensityAnalyzer()

def clean_text(s):
    return str(str(s).encode('ascii','ignore'), 'utf-8').replace('\n', ' ')

for d in nyt_output_list:
    for k, v in d.items():
        if k=='fulltext' or k=='headline':
            d[k]=clean_text(v)

for d in nyt_output_list:
    for k, v in list(d.items()):
        if k=='fulltext':
            d['sentences']=tokenize.sent_tokenize(v)

for d in nyt_output_list:
    for k, v in list(d.items()):
        if k=='headline':
            d['headline_score']=sid.polarity_scores(v)
        if k=='sentences':
            a=[]
            for v in v:
                a.append(sid.polarity_scores(v))  
                d['fulltext_scores'] = a

In [ ]:
for maindict in nyt_output_list:
    compound_list = []
    pos_list =[]
    neg_list =[]
    neu_list =[]
    compound_list_nonzero = []
    pos_list_nonzero =[]
    neg_list_nonzero =[]
    neu_list_nonzero =[]    
    for subdict in maindict['fulltext_scores']:
        compound_list.append(subdict['compound'])
        pos_list.append(subdict['pos'])
        neg_list.append(subdict['neg'])
        neu_list.append(subdict['neu'])
        if subdict['compound']!=0:
            compound_list_nonzero.append(subdict['compound'])
        if subdict['pos']!=0:
            pos_list_nonzero.append(subdict['pos'])
        if subdict['neg']!=0:
            neg_list_nonzero.append(subdict['neg'])
        if subdict['neu']!=0:
            neu_list_nonzero.append(subdict['neu'])  
    
    maindict['mean_compound'] = np.mean(compound_list)
    maindict['median_compound'] = np.median(compound_list)
    maindict['mean_pos'] = np.mean(pos_list)
    maindict['median_pos'] = np.median(pos_list)
    maindict['mean_neg'] = np.mean(neg_list)
    maindict['median_neg'] = np.median(neg_list)
    maindict['mean_neu'] = np.mean(neu_list)
    maindict['median_neu'] = np.median(neu_list)
    
    maindict['mean_compound_nonzero'] = np.mean(compound_list_nonzero)
    maindict['median_compound_nonzero'] = np.median(compound_list_nonzero)
    maindict['mean_pos_nonzero'] = np.mean(pos_list_nonzero)
    maindict['median_pos_nonzero'] = np.median(pos_list_nonzero)
    maindict['mean_neg_nonzero'] = np.mean(neg_list_nonzero)
    maindict['median_neg_nonzero'] = np.median(neg_list_nonzero)
    maindict['mean_neu_nonzero'] = np.mean(neu_list_nonzero)
    maindict['median_neu_nonzero'] = np.median(neu_list_nonzero)

In [ ]:
df2 = pd.DataFrame(nyt_output_list)
df2.to_csv('nyt_output_list_1.csv', index=False)